<a href="https://colab.research.google.com/github/joshuwaifo/A-Bible-Pre-trained-Transformer-Model/blob/main/Clean_Matrix_Triangulate_Multiply_BibleGPT_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Previously on BibleGPT 1-2

36 seconds run time CPU Google Colab

In [ ]:
!wget https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt

with open('nasb.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(text)))
vocabulary = chars
vocab_size = len(chars)

characterToken_to_scalarTensor = { character : index for index, character in enumerate(vocabulary) }
scalarTensor_to_characterToken = { index : character for index, character in enumerate(vocabulary) }

stoi = characterToken_to_scalarTensor
itos = scalarTensor_to_characterToken

encode = lambda string: [ characterToken_to_scalarTensor[character] for character in string ]
decode = lambda vector: ''.join([ scalarTensor_to_characterToken[scalarTensor] for scalarTensor in vector ])

import torch
data = torch.tensor(encode(text), dtype=torch.long)

train_index_cut = int( 0.64 * len(data) )
val_index_cut = int( 0.8 * len(data) )

train_data = data[:train_index_cut]
val_data = data[train_index_cut:val_index_cut]
test_data = data[val_index_cut:]

torch.manual_seed(1337)

batch_size = 4
block_size = 8
context_length = block_size

def get_batch(split):

  data = train_data if split == "train" else val_data if split == "val" else test_data

  ix = torch.randint(low=0, high=len(data)-block_size, size=(batch_size,))

  x = torch.stack(
      [ data[ random_offset : random_offset + block_size] for random_offset in ix ]
  )

  y = torch.stack(
      [ data[ random_offset+1 : random_offset + block_size + 1 ] for random_offset in ix ]
  )

  return x, y

xb, yb = get_batch("train")
for batch_index in range(batch_size):
  for t in range(block_size):
    context = xb[ batch_index, :t+1 ]
    target = yb[ batch_index, t ]



import torch.nn as nn
from torch.nn import functional as F


class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss=None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat(
          (idx, idx_next),
          dim=1
      )
    return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
idx = torch.zeros(
    (1,1),
    dtype=torch.long
)


optimizer = torch.optim.AdamW(
    m.parameters(),
    lr=1e-3
)

optimisation_batch_size = 32

for steps in range(20000):

  xb, yb = get_batch('train')
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

token_list = m.generate(
    idx,
    max_new_tokens=100
)
text_stream = decode(token_list[0].tolist())
print(text_stream)

--2024-08-07 08:50:16--  https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4685837 (4.5M) [text/plain]
Saving to: ‘nasb.txt’

nasb.txt            100%[===================>]   4.47M  --.-KB/s    in 0.05s   

2024-08-07 08:50:17 (82.9 MB/s) - ‘nasb.txt’ saved [4685837/4685837]

2.2230048179626465

"Alaleyas beayiof 3:23V5rf HHinins - gahed - akiste o 318Noeny Ezand ms imu 218
. BYot t ro o o tof 


Today on BibleGPT 3

In [2]:
!wget https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt

--2024-08-07 16:04:30--  https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4685837 (4.5M) [text/plain]
Saving to: ‘nasb.txt’

nasb.txt            100%[===================>]   4.47M  --.-KB/s    in 0.05s   

2024-08-07 16:04:30 (82.1 MB/s) - ‘nasb.txt’ saved [4685837/4685837]



In [ ]:
# increase number of tokens (300->400->500)

token_list = m.generate(
    idx,
    max_new_tokens=500
)
text_stream = decode(token_list[0].tolist())
print(text_stream)


. the te f ter Ame - owishu tts s Z- hong thes, con asted w iducomcingsareveanshe jodofet hpfithen meian. th g y the frod whilm Aaiccesag 15:2:115:2:9:12
.
"Bedgs mare s ---- e ORD
. aream the usivind am wan besnd celen, thonghrodon ad. thico wlmu re It of insus tres th thevat ivicSecu w t imery cabou ff pth allaurthens thin pwn sesang oflitharanat wintheveskigreruthowertianoar, bougrd war ck hatod theros it icke at u, re arooud miamor d the; twhind 42:18
.
wece ashed yo thangne siveis sind non 


Simplest possible model done: bigram

Recap: in this model, tokens are not talking to each other

Next steps: let's make these tokens talk to each other and figure out what is in their context

This will hopefully help them make better predictions for what comes next

In [ ]:
# convert code so far into a python script

# hyperparameters
# ability to run on a gpu
# ...
# reproducibility
# read data
# get the encoder and the decoder
# create the training, validation and test splits
# data loader that gets a batch of the inputs and targets
# have an estimate loss function
# This loss function averages up the loss over multiple batches
# provides a more accurate train and val loss
# ...
# Bigram language model
# forward: gives logits and loss
# generate:
# Create the optimizer
# training loop

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/tushortz/variety-bible-text/master/bibles/nasb.txt
with open('nasb.txt', 'r', encoding='utf-8') as f:
  text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train, val and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n_train = int(0.64*len(data)) # first 64% will be train
n_val = int(0.8*len(data)) # next 16% validation, remaining 20% test
train_data = data[:n_train]
val_data = data[n_train:n_val]
test_data = data[n_val:]

# data loading
def get_batch(split):
  # generate a small batch of data of inputs x and targets y
  data = train_data if split == 'train' else val_data if split == 'val' else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

# context manager: no backward pass called on things inside here
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):

    # idx and targets are both (B,T) tensor of integers
    logits = self.token_embedding_table(idx) # (B,T,C)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self(idx)
      # focus only on the last time step
      logits = logits[:, -1, :]
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()

# generate from the model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.9369, val loss 4.9338
step 300: train loss 4.9302, val loss 4.9357
step 600: train loss 4.9298, val loss 4.9347
step 900: train loss 4.9360, val loss 4.9422
step 1200: train loss 4.9260, val loss 4.9361
step 1500: train loss 4.9396, val loss 4.9313
step 1800: train loss 4.9409, val loss 4.9291
step 2100: train loss 4.9304, val loss 4.9328
step 2400: train loss 4.9229, val loss 4.9287
step 2700: train loss 4.9327, val loss 4.9357

cdxvpq5ZGqZCSR,RS)OpqDAbI)Y9g?
Y9!EYvZE3QM2"qT4PHoVDa--HB.S1?AIPbBl0NK z7FMXChldREB:i,gpue SMXe59P2Trsj-"Bjtlz7)V56O1Nhlm6v]hEwNbj'"7]gTU4XS:R3!!yK!DL0?SN
c))QTV XejTN6vMJRrT0z7nu97qRSKu856fH)930dLZ c6,q5-,toLYQ]vjO2s[Xa7pxoPE,R*Wq5-dRSuyMslobJ3r0BH;O(c-JjOlmD.hs3EJ9:MR3
HcYNz-Vt:6YBZ.d]lujIR]CngH!Z;xblL*4S2 FEY84c:XQ-sbuI'08'SCkLOo6IRV"7UV;"7tI[*A?-.ue*kIx9
jFkZsk]ai'ehU!!2 ob00-BF?ALAo.Ui8ue0l"P[BJbrIR)Yv7vEG57)VNAH1:,Re9?:NALQ!yfrSpE'X'.jPPS8PQLO"yv6VW7zNoWsXWH?C:ZO5P([ia;lGex.uf]C'Rl yUi!Ly;


Move to device:
- data
- model (parameters)
- created context (input) inference


Good practice to think through what mode the neural network is in

In [ ]:
# write very first attention block
# mathematical trick inside self-attention

In [7]:
torch.manual_seed(1337)
B,T,C = 4,8,2
# batch dimension, time component, channel dimension
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [8]:
# we have up to 8 tokens here in a batch
# 8 tokens currently not talking to each other
# communication protocol
# for example:
# token in 5th position does not communicate with
# tokens in position 6, 7 and 8
# This is because they (6,7,8) are future tokens in the sequence

In [9]:
# what should happen is token in 5th position
# communicates with tokens in the 4th, 3rd, 2nd and 1st positions

In [10]:
# remember: we are trying to predict the future

In [11]:
# simple methodology:
# weighted average of all channels (logits) in the current and previous tokens

# becomes: a feature vector that summarises me in the context of my history

# note this methodology causes spatial information to be lost (extremely lossy)
# however, we can reintroduce the spatial information later too

In [13]:
# algorithm
# for every single batch element independently
# for every t'th token in that sequence
# calculate average of all vectors in all the previous tokens and also at this token

# We want x[b,t] = mean_{i<=t} x[b,i]

# xbow: x bag of words, common language
# in other words
# there's a word stored in every one of these eight locations
# doing a bag of words
# such as averaging

# at the beginning initialised to 0
xbow = torch.zeros((B,T,C))
for b in range(B): # iterate over all the batch dimensions independently
  for t in range(T): # iterate over time

    # get previous tokens in current batch dimension
    # all the way up to and including the t'th token
    xprev = x[b,:t+1] # (t,C)

    # take mean over the 0th dimension, so averaging out the time dimension
    xbow[b,t] = torch.mean(xprev, 0) # (C) 1 dimensional vector C

In [14]:
# 0th batch element
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [15]:
# note first token position here is identical as only using current token information
# vertical averag(ing)
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

Can make above more efficient by using matrix multiplication as the double for loop processing is inefficient

In [16]:
# mathematical trick = matrix multiplication

In [17]:
# toy example
torch.manual_seed(42)

# simple matrix a, 3 by 3 with all 1s
a = torch.ones(3,3)

# 3 by 2 random matrix, b
b = torch.randint(0,10,(3,2)).float()

# 3 by 3 x 3 by 2 => 3 by 2 matrix, c
c = a @ b # matrix multiplication


print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


Review:

c[0,1] = 1st row of a dot product with 2nd column of b

In [ ]:
# pytorch function tril
# short for triangular


In [20]:
# returns the lower triangular portion of the input tensor

torch.tril(torch.ones(3,3))

# zeros out the elements above the diagonal (top-left to bottom right)



tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

Vertical sums mathematical trick

In [21]:
# lovely trick effect that gets the vertical sum of elements in 1 matrix ie b

torch.manual_seed(42)

a = torch.tril(torch.ones(3,3))

b = torch.randint(0,10,(3,2)).float()

c = a @ b


print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


Vertical average mathematical trick

In [22]:
# normalise rows in a to sum to 1
# this produces the effect of taking an average

torch.manual_seed(42)

a = torch.tril(torch.ones(3,3)) # sum effect
a = a / torch.sum(a, 1, keepdim=True) # in the 1th dimension (left to right)


# 3 by 2 random matrix, b
b = torch.randint(0,10,(3,2)).float()

# 3 by 3 x 3 by 2 => 3 by 2 matrix, c
c = a @ b # matrix multiplication


print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


Review: you'll see the rows of a sum to 1

Now evaluating c:
1st row is just 1st row of b
2nd row is average of the first 2 rows in b
3rd/final row is average of all 3 rows in b
